In [ ]:
# Install necessary libraries
!pip install crewai crewai_tools
#!pip install google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.3/606.3 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 25.1 MB

In [ ]:
# Import required modules
from crewai import Task, Crew, Agent, Process
from googleapiclient.discovery import build
from crewai.tools import BaseTool

In [ ]:
import requests
from bs4 import BeautifulSoup
from crewai.tools import BaseTool

In [ ]:
GOOGLE_API_KEY = ""
GOOGLE_CSE_ID = ""
import os
os.environ["GEMINI_API_KEY"] = ""

In [ ]:
def google_search(query, api_key=GOOGLE_API_KEY, cse_id=GOOGLE_CSE_ID, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=query, cx=cse_id, **kwargs).execute()
    return res.get('items', [])

def fetch_page_content(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers, timeout=5)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(separator="\n", strip=True)
        return text[:1000] + "..." if len(text) > 1000 else text
    except Exception as e:
        return f"Error fetching page: {e}"

class GoogleSearchAndFetchTool(BaseTool):
    name: str = "Google Search with Page Fetch"
    description: str = "Performs a Google search and fetches a summary of the top 3 result pages."

    def _run(self, query: str) -> str:
        results = google_search(query)
        output = []
        for item in results[:3]:
            title = item.get("title")
            link = item.get("link")
            content = fetch_page_content(link)
            output.append(f"{title}\n{link}\n{content}\n{'-'*40}")
        return "\n".join(output)

# Initialize the custom tool
custom_google_search_tool = GoogleSearchAndFetchTool()

In [ ]:
# Define agents
llm = 'gemini/gemini-2.0-flash'

news_researcher = Agent(
    role='Situational Expert',
    goal='Provide accurate, up-to-date, and practical answers to any real-world question, such as recommendations or current conditions about {topic}.',
    verbose=True,
    memory=True,
    backstory=(
        "You are a resourceful expert, skilled at quickly gathering and analyzing the latest information. "
        "Whether it's about places, events, or current conditions, you deliver clear, reliable, and actionable insights for any question."
    ),
    tools=[custom_google_search_tool],  # Use the initialized custom tool
    llm=llm,
    allow_delegation=True
)

news_writer = Agent(
    role='Conversational Communicator',
    goal='Present information in a friendly, clear, and engaging way, making it easy for anyone to understand and act on answers about {topic}.',
    verbose=True,
    memory=True,
    backstory=(
        "You specialize in turning facts and recommendations into helpful, easy-to-read responses. "
        "Your writing is approachable and practical, perfect for answering real-world questions people care about."
    ),
    #tools=[tool],
    llm=llm,
    allow_delegation=False
)

In [ ]:
# Define tasks
research_task = Task(
    description=(
        "Research and gather the most accurate, up-to-date, and practical information about: {topic}. "
        "Focus on providing clear answers, recommendations, or current conditions relevant to the question. "
        "Include details that help someone make decisions or understand the situation right now."
    ),
    expected_output=(
    "A concise summary with key facts, recommendations, or current status about {topic}, "
    "suitable for someone seeking real-world, actionable information."
    ),
    tools=[custom_google_search_tool], # Use the initialized custom tool
    agent=news_researcher,
)

write_task = Task(
    description=(
    "Based on the research, write a clear and friendly response about {topic}. "
    "Make the answer easy to understand, practical, and directly useful for the question asked. "
    "If relevant, include tips, examples, or next steps."
    ),
    expected_output=(
    "A short, well-structured answer (2-4 paragraphs) in markdown format, "
    "providing practical and up-to-date information about {topic}."
    ),
    #tools=[tool],
    agent=news_writer,
    async_execution=False,
    output_file='new-blog-post.md'
)

In [ ]:
# Initialize the crew
crew = Crew(
    agents=[news_researcher, news_writer],
    tasks=[research_task, write_task],
    process=Process.sequential,
)
import pprint
# Start the task execution process
result = crew.kickoff(inputs={'topic': 'Read from https://vedabase.io/en/library/bg/4/11/ and summarize it as bullet points'})
print(result)

# Agent: Situational Expert
## Task: Research and gather the most accurate, up-to-date, and practical information about: Read from https://vedabase.io/en/library/bg/4/11/ and summarize it as bullet points. Focus on providing clear answers, recommendations, or current conditions relevant to the question. Include details that help someone make decisions or understand the situation right now.


# Agent: Situational Expert
## Thought: I need to summarize the content of the webpage https://vedabase.io/en/library/bg/4/11/ as bullet points. I will use the Google Search with Page Fetch tool to get the content of the page, then I will summarize the content as bullet points.
## Using tool: Google Search with Page Fetch
## Tool Input: 
"{\"query\": \"vedabase.io Bhagavad-gita 4.11\"}"
## Tool Output: 
Bhagavad-gītā As It Is 4.11
https://vedabase.io/en/library/bg/4/11/
Bhagavad-gītā As It Is 4.11
Bhaktivedanta
Vedabase
Settings
English
en
Library
Search
Support Us
Tools
Contact
Library
»
Bhagavad-

In [ ]:
pprint.pprint(result)

CrewOutput(raw='Craving authentic Hyderabadi Biryani? You\'re in for a treat! Hyderabad is famous for its flavorful Biryani, and here are some of the best spots to get your fix:\n\n*   **Bawarchi (RTC Cross Road, Nallakunta):** This place is legendary! Known for consistently delicious and aromatic Biryani, generous portions, and a lively atmosphere. Expect a bit of a crowd, but it\'s worth the wait.\n*   **Paradise Restaurant (Multiple locations):** Perhaps the most iconic Biryani restaurant in Hyderabad. With several locations across the city, Paradise offers consistently high-quality Hyderabadi Dum Biryani. It\'s a reliable choice for both locals and tourists.\n*   **Shah Ghouse Cafe & Restaurant (Multiple locations):** A favorite for late-night Biryani cravings! Shah Ghouse is known for its spicy and flavorful Biryani, perfect for those who like a bit of heat.\n*   **Cafe Bahar (Basheerbagh):** A long-standing establishment popular for family dinners. Their Biryani stands out with i